# # Visualización de respuestas

Importaciones

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import yaml

Conexión a la base de datos

In [2]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

Cargar las tablas de hechos del etl

In [3]:
df_hecho_novedades = pd.read_sql_table('hecho_novedades', etl_conn)
df_hecho_acumulating_servicios = pd.read_sql_table('hecho_acumulating_servicios', etl_conn)
df_hecho_servicios_dia = pd.read_sql_table('hecho_servicios_dia', etl_conn)
df_hecho_servicios_hora = pd.read_sql_table('hecho_servicios_hora', etl_conn)

Cargar las tablas de dimensiones del etl

In [4]:
df_dim_cliente = pd.read_sql_table('dim_cliente', etl_conn)
df_dim_fecha = pd.read_sql_table('dim_fecha', etl_conn)
df_dim_hora = pd.read_sql_table('dim_hora', etl_conn)
df_dim_mensajero = pd.read_sql_table('dim_mensajero', etl_conn)
df_dim_novedad = pd.read_sql_table('dim_novedad', etl_conn)
df_dim_sede = pd.read_sql_table('dim_sede', etl_conn)

# Verificar la cantidad de registros en las tablas de hechos

In [38]:
print('Fact Tables')
print('Hecho Novedades:', df_hecho_novedades.shape[0])
print('Hecho Acumulating Servicios:', df_hecho_acumulating_servicios.shape[0])
print('Hecho Servicios Dia:', df_hecho_servicios_dia.shape[0])
num_servicios_hora = df_hecho_servicios_hora['numero_servicios'].sum()
print('Hecho Servicios Hora:', num_servicios_hora)
num_servicios_dia = df_hecho_servicios_dia['numero_servicios'].sum()
print('Hecho Servicios Dia:', num_servicios_dia, '\n')

Fact Tables
Hecho Novedades: 5208
Hecho Acumulating Servicios: 26365
Hecho Servicios Dia: 250
Hecho Servicios Hora: 26365
Hecho Servicios Dia: 26365 



# 1) En qué meses del año los clientes solicitan más servicios de mensajería

In [39]:
print('Meses con más solicitudes de servicios de mensajería')
response_1 = df_hecho_servicios_dia.sort_values(by='numero_servicios', ascending=False)
response_1 = response_1.merge(
    df_dim_fecha,
    left_on='key_dim_fecha_solicitud',
    right_on='key_dim_fecha')[['nombre_mes', 'mes', 'numero_servicios']]
response_1 = response_1.groupby(['nombre_mes', 'mes']).sum().sort_values(by='numero_servicios', ascending=False)
print(response_1, '\n')

Meses con más solicitudes de servicios de mensajería
                numero_servicios
nombre_mes mes                  
May        5                4409
July       7                4298
April      4                4095
August     8                4083
June       6                3896
March      3                3073
February   2                2201
January    1                 262
December   12                 22
September  9                  13
November   11                  8
October    10                  5 



# 2) ¿Cuáles son los días dónde más solicitudes hay?

In [40]:
print('Días con más solicitudes de servicios de mensajería')
response_2 = df_hecho_servicios_dia.sort_values(by='numero_servicios', ascending=False)
response_2 = response_2.merge(
    df_dim_fecha,
    left_on='key_dim_fecha_solicitud',
    right_on='key_dim_fecha')[['numero_servicios', 'dia', 'nombre_dia']]
print(response_2, '\n')


Días con más solicitudes de servicios de mensajería
     numero_servicios  dia nombre_dia
0                 230   14    Tuesday
1                 216   26     Monday
2                 213   17     Friday
3                 212   30     Friday
4                 207   21    Tuesday
..                ...  ...        ...
245                 1   23     Monday
246                 1   20     Friday
247                 1   13     Friday
248                 1    3    Tuesday
249                 1   26    Tuesday

[250 rows x 3 columns] 



# 3) ¿A qué hora los mensajeros están más ocupados?

In [41]:
print('Horas con más solicitudes de servicios de mensajería')
response_3 = df_hecho_servicios_hora.sort_values(by='numero_servicios', ascending=False)
response_3 = response_3.merge(
    df_dim_hora,
    left_on='key_dim_hora_solicitud',
    right_on='key_dim_hora')[['hora', 'numero_servicios']]
print(response_3, '\n')

Horas con más solicitudes de servicios de mensajería
   hora  numero_servicios
0    09              3184
1    11              3086
2    10              2735
3    08              2474
4    15              2405
5    14              2369
6    16              1875
7    12              1604
8    13              1382
9    17              1226
10   07              1001
11   06               873
12   18               404
13   19               318
14   20               208
15   21               163
16   01               156
17   23               151
18   02               147
19   22               140
20   00               135
21   05               128
22   03               102
23   04                99 



# 4) ¿Cuántos servicios ha solicitado cada cliente por mes?

In [42]:
print('Número de servicios solicitados por cliente y por mes')
response_4 = df_hecho_acumulating_servicios.merge(
    df_dim_cliente,
    left_on='key_dim_cliente',
    right_on='key_dim_cliente').merge(
    df_dim_fecha,
    left_on='key_dim_fecha_solicitud',
    right_on='key_dim_fecha')

response_4 = response_4.groupby(['cliente_id', 'nombre', 'nombre_mes']).size().reset_index(name='numero_servicios')
print(response_4, '\n')


Número de servicios solicitados por cliente y por mes
     cliente_id             nombre nombre_mes  numero_servicios
0             2          Cliente 1      April                 1
1             2          Cliente 1   February                 4
2             2          Cliente 1    January                 1
3             2          Cliente 1       June                 1
4             2          Cliente 1      March                 2
..          ...                ...        ...               ...
98           25  CALI SULUD Y VIDA       July               169
99           25  CALI SULUD Y VIDA       June               147
100          25  CALI SULUD Y VIDA      March               114
101          25  CALI SULUD Y VIDA        May               150
102          27    MEDICOS Y SALUD     August                 1

[103 rows x 4 columns] 



# 5) Mensajeros más eficientes (Los que más servicios prestan)

In [43]:
print('Mensajeros más eficientes')
response_5 = df_hecho_acumulating_servicios.merge(
    df_dim_mensajero,
    left_on='key_dim_mensajero',
    right_on='key_dim_mensajero').groupby(['mensajero_id', 'username']).size().reset_index(name='numero_servicios')
response_5 = response_5.sort_values(by='numero_servicios', ascending=False)
print(response_5, '\n')


Mensajeros más eficientes
    mensajero_id           username  numero_servicios
22            30        JHONTROCHEZ              2402
21            29    Sebastian Acuña              1438
9             15        Juan Solano              1425
18            25    ANDRESGUTIERREZ              1392
10            16         Jan Sastre              1303
23            31     Hector Aquiles              1271
32            41        LUISCARDONA              1269
33            42           JPEDROZA              1214
15            22      Jairon Montes              1191
20            28          JHONMUÑOZ              1159
6             11  Carlos Valderrama              1055
19            27  ALEXANDERMOSQUERA              1027
4              8        Luis Castro               957
12            18     Nicolas Maduro               911
0              3         Biil-Gates               833
35            44           RVALLEJO               813
26            34       VICTORARARAT               700
24

# 6) ¿Cuáles son las sedes que más servicios solicitan por cada cliente?

In [44]:
print('Sedes que más servicios solicitan por cada cliente')
response_6 = df_hecho_acumulating_servicios.merge(
    df_dim_sede,
    left_on='key_dim_sede',
    right_on='key_dim_sede').merge(
    df_dim_cliente,
    left_on='key_dim_cliente',
    right_on='key_dim_cliente').groupby(['cliente_id', 'nombre', 'sede_id', 'sede_nombre']).size().reset_index(name='numero_servicios')
response_6 = response_6.sort_values(by=['cliente_id', 'numero_servicios'], ascending=False)
print(response_6, '\n')

Sedes que más servicios solicitan por cada cliente
    cliente_id                                   nombre  sede_id  \
40          27                          MEDICOS Y SALUD       85   
39          25                        CALI SULUD Y VIDA       45   
38          25                        CALI SULUD Y VIDA       44   
37          24          CLINICA SAN RAFAEL DE OCCIDENTE       46   
36          22  FUNDACION CLINICA INFANTIL LOS GRITONES       51   
34          12                  CLINICA DEL SUR DE CALI       32   
35          12                  CLINICA DEL SUR DE CALI       48   
19          11             CARROS DEL  PACIFICO (CHINA)       15   
21          11             CARROS DEL  PACIFICO (CHINA)       17   
30          11             CARROS DEL  PACIFICO (CHINA)       26   
20          11             CARROS DEL  PACIFICO (CHINA)       16   
22          11             CARROS DEL  PACIFICO (CHINA)       18   
31          11             CARROS DEL  PACIFICO (CHINA)       33 

# 7) ¿Cuál es el tiempo promedio de entrega desde que se solicita el servicio hasta que se cierra el caso?

In [45]:
print('Tiempo promedio de entrega \n')
# make average of total_tiempo_minutos and total_tiempo_horas
response_7 = df_hecho_acumulating_servicios
avg_tiempo_minutos = round(response_7['total_tiempo_minutos'].mean(), 2)
avg_tiempo_horas = round(response_7['total_tiempo_horas'].mean(), 2)
print('Tiempo promedio en minutos:', avg_tiempo_minutos)
print('Tiempo promedio en horas:', avg_tiempo_horas, '\n')

Tiempo promedio de entrega 

Tiempo promedio en minutos: 404.42
Tiempo promedio en horas: 6.74 



# 8) Mostrar los tiempos de espera por cada fase del servicio: Iniciado, Con mensajero asignado, recogido en origen, Entregado en Destino, Cerrado. En que fase del servicio hay más demoras?

In [46]:
print('Tiempo de espera promedio por cada fase del servicio')
response_8 = df_hecho_acumulating_servicios

def get_avg_time(df, column):
    return round(df[column].mean(), 2)

response_8_mins = {
    'Asignacion minutos': get_avg_time(response_8, 'tiempo_minutos_asignacion'),
    'Recogido minutos': get_avg_time(response_8, 'tiempo_minutos_recogida'),
    'Entregado minutos': get_avg_time(response_8, 'tiempo_minutos_entrega'),
    'Cerrado minutos': get_avg_time(response_8, 'tiempo_minutos_cerrado'),
}

response_8_hours = {
    'Asignacion horas': get_avg_time(response_8, 'tiempo_horas_asignacion'),
    'Recogido horas': get_avg_time(response_8, 'tiempo_horas_recogida'),
    'Entregado horas': get_avg_time(response_8, 'tiempo_horas_entrega'),
    'Cerrado horas': get_avg_time(response_8, 'tiempo_horas_cerrado')
}

for key, value in response_8_mins.items():
    print(key, value)
for key, value in response_8_hours.items():
    print(key, value)
# get the max value
max_value_mins = max(response_8_mins, key=response_8_mins.get)
max_value_hours = max(response_8_hours, key=response_8_hours.get)
print('Fase con más demoras en minutos:', max_value_mins, " Tiempo: ", response_8_mins[max_value_mins])
print('Fase con más demoras en horas:', max_value_hours, " Tiempo: ", response_8_hours[max_value_hours])

Tiempo de espera promedio por cada fase del servicio
Asignacion minutos 147.41
Recogido minutos 85.76
Entregado minutos 87.11
Cerrado minutos 84.15
Asignacion horas 2.46
Recogido horas 1.43
Entregado horas 1.45
Cerrado horas 1.4
Fase con más demoras en minutos: Asignacion minutos  Tiempo:  147.41
Fase con más demoras en horas: Asignacion horas  Tiempo:  2.46


# 9) ¿Cuáles son las novedades que más se presentan durante la prestación del servicio?

In [48]:
print('Novedades que más se presentan durante la prestación del servicio')
response_9 = df_hecho_novedades.merge(
    df_dim_novedad,
    left_on='key_dim_novedad',
    right_on='key_dim_novedad').groupby(['tipo_novedad']).size().reset_index(name='numero_novedades')
response_9 = response_9.sort_values(by='numero_novedades', ascending=False)
print(response_9, '\n')

Novedades que más se presentan durante la prestación del servicio
             tipo_novedad  numero_novedades
1  Novedades del servicio              3892
0      No puedo continuar              1316 

